In [40]:
import numpy as np
import pandas as pd

In [41]:
SNFs = pd.read_excel('Data/SNFs.xlsx')

In [42]:
Episodes = pd.read_excel('Data/Episodes.xlsx')

In [43]:
SNFs.head()

,ProviderName,PID,ProviderPracticeAddress1,ProviderPracticeAddress2,ProviderPracticeCity,ProviderPracticeStateCode,ProviderPracticeZipCode,County
0,648 Medical Squadron,1,8006 CHENNAULT RD,STE 1,BROOKS CITY BASE,TX,78235,Bexar
1,Abilene Nursing and Rehabilitation Center,2,2630 OLD ANSON RD,NaN,ABILENE,TX,79603,Taylor
2,Abilene Regional Medical Center,3,6250 US HIGHWAY 83,NaN,ABILENE,TX,79606,Taylor
3,Abilene Regional Medical Center Skilled Nursin...,4,6250 HIGHWAY 83 84 AT ANTILLEY RD,NaN,ABILENE,TX,79606,Taylor
4,"Absolutely Angels, Inc",5,936 E PRODUCTION DR,NaN,PILOT POINT,TX,76258,Denton


In [44]:
Episodes.head()

,PID,EpisodeID,ContractType,AdmitDate,DischargeDate
0,530,1,CT1,12/1/2019,2020-01-30
1,1107,2,CT1,12/1/2019,NaT
2,1107,3,CT2,12/1/2019,2019-12-07
3,1107,4,CT2,12/1/2019,2019-12-02
4,1249,5,CT1,12/1/2019,NaT


In [45]:
# merge SNFs with Episodes and only keep SNF that has patient data in Episodes
# Reasons to do this: simplier question and, for the SNFs that do not have any patient record,
# no way to confirm that they need CCs and they need how many CCs.
common_epi_snf = SNFs.merge(Episodes, on=["PID"])
SNFs_no_episodes = SNFs[~SNFs.PID.isin(common_epi_snf.PID)]

In [46]:
common_snf = SNFs.merge(SNFs_no_episodes, on=["PID"])
SNFs_with_episodes = SNFs[~SNFs.PID.isin(common_snf.PID)]

In [47]:
# Align with unique address and city
repeat = SNFs_with_episodes[SNFs_with_episodes.duplicated(subset=['ProviderPracticeAddress1', 'ProviderPracticeCity'],keep=False)].sort_values(by ='ProviderPracticeAddress1')

In [48]:
common_both = SNFs_with_episodes.merge(repeat, on=["PID"])
SNFs_unique = SNFs_with_episodes[~SNFs_with_episodes.PID.isin(common_both.PID)]

In [49]:
repeat_for_join = repeat.groupby(['ProviderPracticeAddress1','County'], as_index = False).first()

In [50]:
frames = [SNFs_unique, repeat_for_join]
SNFs_fixed = pd.concat(frames, sort = False)

In [51]:
SNFs_fixed.to_csv('Data/SNFs.csv', index=False)

In [52]:
#Replacing wrong PIDs
Episodes['PID'] = Episodes['PID'].replace([560, 1770, 1632, 484, 147, 463, 804, 474, 837, 757, 135, 612, 16],[561, 1771, 1631, 485, 146, 464, 802, 478, 836, 38, 138, 613, 12])

In [53]:
# convert to datetime type
Episodes['AdmitDate']= pd.to_datetime(Episodes['AdmitDate'])
Episodes['DischargeDate']= pd.to_datetime(Episodes['DischargeDate'])

In [54]:
Episodes = Episodes.dropna(axis=0, subset=['DischargeDate'])

In [55]:
# have a new column store time difference
Episodes['time_diff'] = Episodes['DischargeDate'] - Episodes['AdmitDate']

In [56]:
Episodes = pd.DataFrame(Episodes)

In [57]:
Episodes = Episodes[Episodes['time_diff'].dt.days >= 0]

In [58]:
Episodes = Episodes.drop(['time_diff'], axis = 1)

In [59]:
Episodes.to_csv('Data/Episodes.csv', index=False)